<a href="https://colab.research.google.com/github/sandeepaamcp/privacy-metrics/blob/main/Differential_Privacy_Evaluation_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Differential Privacy evalution for ML models

This notebook uses Opacus library for adding noise to the gradients via Differential Privacy (DP). First, required libraries are installed. Then, the configurations for DP is stored as Args. Next, dataset and NN architectures are made. The plain model is sent to a privacy engine, which will implement DP.

### Install libraries

In [1]:
!pip install opacus --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 13.3 MB/s eta 0:00:00


In [2]:
!pip install adversarial-robustness-toolbox --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 24.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from opacus import PrivacyEngine
from torchvision import datasets, transforms
from tqdm import tqdm
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, SubsetRandomSampler


In [4]:
# Precomputed characteristics of the MNIST dataset
MNIST_MEAN = 0.1307
MNIST_STD = 0.3081

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Args:
    epochs=15 # number of epochs
    n_runs=1 # allow multiple independent run so you can take average of accuracy
    lr=0.25 # learning rate
    sigma=1.3 # noise multiplier in the DP optimizer, necessary to set to non-zero for DP
    max_per_sample_grad_norm=1.5  # clipping norm to guarantee the sensitivity of gradients; necessary to set to non-infinity for DP
    delta=1e-5 # delta in (epsilon, delta)-DP, need to be smaller than 1/sample size
    data_root="./data"
    disable_dp=False
    batch_size=64
    test_batch_size=1024
    secure_rng=False
    epsilon=1
    # if True, then this is training with no clipping (hence not DP);
    # if False, and if sigma not zero, then DP; else, this is non-DP training with clipping

args = Args()

In [6]:
class SampleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

    def name(self):
        return "SampleConvNet"

In [7]:
def train(args, model, device, train_loader, optimizer, privacy_engine, epoch):
    # sample function to get length of the dataset
    num_samples = 0
    for batch in train_loader:
        num_samples += batch[0].shape[0]

    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    correct = 0

    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    if not args.disable_dp:
        epsilon = privacy_engine.accountant.get_epsilon(delta=args.delta)
        print(
            f"Train Epoch: {epoch} \t"
            f"Loss: {np.mean(losses):.6f} "
            f"Accuracy: {correct/num_samples:.6f} "
            f"(ε = {epsilon:.2f}, δ = {args.delta})"
        )
    else:
        # print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f}")
        print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f} \t Accuracy: {correct/num_samples:.6f}")

    return correct/num_samples, np.mean(losses)

In [8]:
def test(model, device, test_loader):
    num_samples = 0
    for batch in test_loader:
        num_samples += batch[0].shape[0]

    model.eval()
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= num_samples

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            num_samples,
            100.0 * correct / num_samples,
        )
    )
    return correct / num_samples

In [9]:
train_examples = 1000
test_examples = 1000
batch_size = 100

mnist_train = MNIST(root="./data", train=True, download=True,transform=transforms.Compose(
                       [transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081)),]))
mnist_test = MNIST(root="./data", train=False, download=True,transform=transforms.Compose(
                       [transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081)),]))

# Create a subset sampler that selects num_examples randomly from the dataset
indices_train = torch.randperm(len(mnist_train))[:train_examples]
sampler_train = SubsetRandomSampler(indices_train)
indices_test = torch.randperm(len(mnist_test))[:test_examples]
sampler_test = SubsetRandomSampler(indices_test)

train_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, sampler=sampler_train)
test_loader = DataLoader(dataset=mnist_test, batch_size=batch_size, sampler=sampler_test)

100%|██████████| 9912422/9912422 [00:00<00:00, 191983298.98it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 30729501.22it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 59841579.97it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17904632.30it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Wrapping with DP engine

In [10]:
from art.estimators.classification.pytorch import PyTorchClassifier

run_results = []
args.disable_dp=True # considering no DP
for _ in range(args.n_runs):
    model = SampleConvNet().to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0)
    privacy_engine = None

    if not args.disable_dp:
        privacy_engine = PrivacyEngine(secure_mode=args.secure_rng)
        model, optimizer, train_loader = privacy_engine.make_private(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier=args.sigma,
            max_grad_norm=args.max_per_sample_grad_norm,
        )

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, privacy_engine, epoch)
    run_results.append(test(model, device, test_loader))

if len(run_results) > 1:
    print(
        "Accuracy averaged over {} runs: {:.2f}% ± {:.2f}%".format(
            len(run_results), np.mean(run_results) * 100, np.std(run_results) * 100
        )
    )

100%|██████████| 10/10 [00:09<00:00,  1.04it/s]


Train Epoch: 1 	 Loss: 2.254601 	 Accuracy: 0.150000


100%|██████████| 10/10 [00:00<00:00, 41.94it/s]


Train Epoch: 2 	 Loss: 2.266713 	 Accuracy: 0.265000


100%|██████████| 10/10 [00:00<00:00, 39.42it/s]


...

Train Epoch: 14 	 Loss: 0.147573 	 Accuracy: 0.949000


100%|██████████| 10/10 [00:00<00:00, 41.36it/s]


Train Epoch: 15 	 Loss: 0.200132 	 Accuracy: 0.930000


100%|██████████| 10/10 [00:00<00:00, 42.03it/s]


Test set: Average loss: 0.0145, Accuracy: 701/1000 (70.10%)



## Testing the model accuracy

In [11]:
x_train = []
y_train = []
for batch in train_loader:
    data, labels = batch
    x_train.append(data.numpy())
    y_train.append(labels.numpy())
x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

In [12]:
x_test = []
y_test = []
for batch in test_loader:
    data, labels = batch
    x_test.append(data.numpy())
    y_test.append(labels.numpy())
x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [13]:
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'

criterion = nn.CrossEntropyLoss().to(device)

optimizer_ori =  optim.SGD(model.parameters(), lr=args.lr, momentum=0)

mlp_art_model = PyTorchClassifier(model=model, loss=criterion, optimizer=optimizer_ori, input_shape=(1, 28, 28), nb_classes=10)

train_pred = np.array([np.argmax(arr) for arr in mlp_art_model.predict(x_train.astype(np.float32))])
print('Base model Train accuracy: ', np.sum(train_pred == y_train) / len(y_train))

test_pred = np.array([np.argmax(arr) for arr in mlp_art_model.predict(x_test.astype(np.float32))])
print('Base model Test accuracy: ', np.sum(test_pred == y_test) / len(y_test))

Base model Train accuracy:  0.74
Base model Test accuracy:  0.701


### Using Epsilon for DP

In [14]:
run_results = []
for _ in range(args.n_runs):
    model = SampleConvNet().to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0)
    privacy_engine = None

    if not args.disable_dp:
        privacy_engine = PrivacyEngine(secure_mode=args.secure_rng)
        model, optimizer, train_loader = privacy_engine.make_private(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier=args.sigma,
            max_grad_norm=args.max_per_sample_grad_norm,
        )
        model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        epochs=args.epochs+1,
        target_epsilon=args.epsilon,
        target_delta=args.delta,
        max_grad_norm=args.max_per_sample_grad_norm,
    )


    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, privacy_engine, epoch)
    run_results.append(test(model, device, test_loader))

if len(run_results) > 1:
    print(
        "Accuracy averaged over {} runs: {:.2f}% ± {:.2f}%".format(
            len(run_results), np.mean(run_results) * 100, np.std(run_results) * 100
        )
    )

100%|██████████| 10/10 [00:00<00:00, 25.30it/s]


Train Epoch: 1 	 Loss: 2.214102 	 Accuracy: 0.168000


100%|██████████| 10/10 [00:00<00:00, 38.80it/s]


Train Epoch: 2 	 Loss: 2.109853 	 Accuracy: 0.289000


100%|██████████| 10/10 [00:00<00:00, 39.26it/s]


...

Train Epoch: 14 	 Loss: 1.411424 	 Accuracy: 0.519000


100%|██████████| 10/10 [00:00<00:00, 23.80it/s]


Train Epoch: 15 	 Loss: 1.060936 	 Accuracy: 0.630000


100%|██████████| 10/10 [00:00<00:00, 24.67it/s]


Test set: Average loss: 0.0096, Accuracy: 679/1000 (67.90%)



## Experiment with multiple dataset sizes

In [15]:
def get_train_and_test_loaders(train_examples,test_examples=50,batch_size=10):

  batch_size = min(batch_size,train_examples)

  mnist_train = MNIST(root="./data", train=True, download=True,transform=transforms.Compose(
                        [transforms.ToTensor(),
                          transforms.Normalize((0.1307), (0.3081)),]))
  mnist_test = MNIST(root="./data", train=False, download=True,transform=transforms.Compose(
                        [transforms.ToTensor(),
                          transforms.Normalize((0.1307), (0.3081)),]))

  # Create a subset sampler that selects num_examples randomly from the dataset
  indices_train = np.random.choice(len(mnist_train), train_examples, replace=False)
  indices_test = np.random.choice(len(mnist_test), test_examples, replace=False)
  train_set_small = torch.utils.data.Subset(mnist_train, indices_train)
  test_set_small = torch.utils.data.Subset(mnist_test, indices_test)

  train_loader = DataLoader(dataset=train_set_small, batch_size=batch_size)
  test_loader = DataLoader(dataset=mnist_test, batch_size=batch_size)
  return train_loader,test_loader

In [16]:
train,test = get_train_and_test_loaders(100, 50,10)

In [17]:
def train(args, model, device, train_loader, optimizer, privacy_engine, epoch):
    # sample function to get length of the dataset
    num_samples = 0
    for batch in train_loader:
        num_samples += batch[0].shape[0]

    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    correct = 0

    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    if not args.disable_dp:
        epsilon = privacy_engine.accountant.get_epsilon(delta=args.delta)
        print(
            f"Train Epoch: {epoch} \t"
            f"Loss: {np.mean(losses):.6f} "
            f"Accuracy: {correct/num_samples:.6f} "
            f"(ε = {epsilon:.2f}, δ = {args.delta})"
        )
    else:
        print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f}")
        print(f"Train Epoch: {epoch} \t Loss: {np.mean(losses):.6f} \t Accuracy: {correct/num_samples:.6f}")

    return correct/num_samples, np.mean(losses)

### Analysing the impact of different training set sizes

In [18]:
def run_exp(train_datset_len_list):
  all_train_acc_list = []
  all_test_acc_list = []
  for data_len in train_dataset_len_list:
    run_results = []
    train_results = []
    train_loader, test_loader = get_train_and_test_loaders(train_examples=data_len)
    for _ in range(args.n_runs):
        model = SampleConvNet().to(device)
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0)
        privacy_engine = None

        if not args.disable_dp:
            privacy_engine = PrivacyEngine(secure_mode=args.secure_rng)
            model, optimizer, train_loader = privacy_engine.make_private(
                module=model,
                optimizer=optimizer,
                data_loader=train_loader,
                noise_multiplier=args.sigma,
                max_grad_norm=args.max_per_sample_grad_norm,
            )
            model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            epochs=args.epochs+1,
            target_epsilon=args.epsilon,
            target_delta=args.delta,
            max_grad_norm=args.max_per_sample_grad_norm,
        )


        for epoch in range(1, args.epochs + 1):
            train(args, model, device, train_loader, optimizer, privacy_engine, epoch)
        test(model, device, test_loader)

    all_train_acc_list.append(train_results)
    all_test_acc_list.append(run_results)
    if len(run_results) > 1:
        print(
            "Accuracy averaged over {} runs: {:.2f}% ± {:.2f}% for dataset len{}".format(
                len(run_results), np.mean(run_results) * 100, np.std(run_results) * 100, data_len
            )
        )
  return all_train_acc_list,all_test_acc_list

In [19]:
def test(model, device, test_loader):
    num_samples = 0
    for batch in test_loader:
        num_samples += batch[0].shape[0]

    model.eval()
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= num_samples

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            num_samples,
            100.0 * correct / num_samples,
        )
    )
    return correct / num_samples

In [20]:
train_dataset_len_list = [15]
args.epsilon=1

all_train_acc_list_2,all_test_acc_list_2=run_exp(train_dataset_len_list)

100%|██████████| 2/2 [00:00<00:00, 60.69it/s]


Train Epoch: 1 	 Loss: 2.309582
Train Epoch: 1 	 Loss: 2.309582 	 Accuracy: 0.200000


100%|██████████| 2/2 [00:00<00:00, 187.65it/s]


Train Epoch: 2 	 Loss: 2.417897
Train Epoch: 2 	 Loss: 2.417897 	 Accuracy: 0.066667


100%|██████████| 2/2 [00:00<00:00, 192.67it/s]


...

Train Epoch: 14 	 Loss: 1.973847
Train Epoch: 14 	 Loss: 1.973847 	 Accuracy: 0.266667


100%|██████████| 2/2 [00:00<00:00, 248.60it/s]


Train Epoch: 15 	 Loss: 2.026254
Train Epoch: 15 	 Loss: 2.026254 	 Accuracy: 0.066667


100%|██████████| 1000/1000 [00:03<00:00, 325.02it/s]


Test set: Average loss: 0.2608, Accuracy: 982/10000 (9.82%)



In [ ]:
train_dataset_len_list = [20]
args.epsilon=1

all_train_acc_list_3,all_test_acc_list_3=run_exp(train_dataset_len_list)

/usr/local/lib/python3.9/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some 

Train Epoch: 1 	Loss: 2.301947 Accuracy: 0.150852 (ε = 0.24, δ = 1e-05)


100%|██████████| 2/2 [00:13<00:00,  6.82s/it]


Train Epoch: 2 	Loss: 2.281597 Accuracy: 0.184027 (ε = 0.33, δ = 1e-05)


100%|██████████| 2/2 [00:14<00:00,  7.07s/it]


...

Train Epoch: 14 	Loss: 1.824483 Accuracy: 0.596564 (ε = 0.93, δ = 1e-05)


100%|██████████| 2/2 [00:13<00:00,  6.83s/it]


Train Epoch: 15 	Loss: 1.754239 Accuracy: 0.612885 (ε = 0.96, δ = 1e-05)


100%|██████████| 5/5 [00:00<00:00, 210.35it/s]


Test set: Average loss: 0.1641, Accuracy: 36/50 (72.00%)



In [ ]:
train_dataset_len_list = [50]
args.epsilon=1

all_train_acc_list_2,all_test_acc_list_2=run_exp(train_dataset_len_list)

100%|██████████| 5/5 [00:13<00:00,  2.72s/it]


Train Epoch: 1 	Loss: 2.292116 Accuracy: 0.122305 (ε = 0.24, δ = 1e-05)


100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


Train Epoch: 2 	Loss: 2.253253 Accuracy: 0.283089 (ε = 0.34, δ = 1e-05)


100%|██████████| 5/5 [00:13<00:00,  2.71s/it]


...

Train Epoch: 14 	Loss: 0.917207 Accuracy: 0.740609 (ε = 0.93, δ = 1e-05)


100%|██████████| 5/5 [00:13<00:00,  2.78s/it]


Train Epoch: 15 	Loss: 0.850944 Accuracy: 0.754029 (ε = 0.96, δ = 1e-05)


100%|██████████| 5/5 [00:00<00:00, 154.20it/s]


Test set: Average loss: 0.0734, Accuracy: 37/50 (74.00%)



In [ ]:
train_dataset_len_list = [100]
args.epsilon=1

all_train_acc_list_2,all_test_acc_list_2=run_exp(train_dataset_len_list)

100%|██████████| 10/10 [00:14<00:00,  1.47s/it]


Train Epoch: 1 	Loss: 2.261874 Accuracy: 0.160532 (ε = 0.24, δ = 1e-05)


100%|██████████| 10/10 [00:14<00:00,  1.41s/it]


Train Epoch: 2 	Loss: 2.124238 Accuracy: 0.381287 (ε = 0.34, δ = 1e-05)


100%|██████████| 10/10 [00:14<00:00,  1.41s/it]


...

Train Epoch: 14 	Loss: 0.468459 Accuracy: 0.861254 (ε = 0.93, δ = 1e-05)


100%|██████████| 10/10 [00:14<00:00,  1.40s/it]


Train Epoch: 15 	Loss: 0.452052 Accuracy: 0.854296 (ε = 0.96, δ = 1e-05)


100%|██████████| 5/5 [00:00<00:00, 222.02it/s]


Test set: Average loss: 0.0374, Accuracy: 42/50 (84.00%)

